In [ ]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: []


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list  # Test if it works

In [ ]:
# Download the dataset
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification

# Verify download
!ls -lh

In [ ]:
import zipfile
import os

# Extract
with zipfile.ZipFile('fruits-fresh-and-rotten-for-classification.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/fruit_dataset/')

# Check structure
!ls -R /content/fruit_dataset/

In [1]:
# ==========================================
# FOOD FRESHNESS CLASSIFICATION USING RESNET50
# ==========================================
# Student: [Your Name]
# Part: Supervised Learning with ResNet50
# Dataset: Fruits Fresh and Rotten for Classification
# ==========================================

# STEP 1: Import Required Libraries
# ==========================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os
from google.colab import drive
import zipfile
import cv2
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

# ==========================================
# STEP 2: Mount Google Drive and Setup Kaggle
# ==========================================
# Mount Google Drive
drive.mount('/content/drive')

# Setup Kaggle API (Upload your kaggle.json to Colab)
# Instructions: Download kaggle.json from Kaggle Account Settings
from google.colab import files
print("Please upload your kaggle.json file:")
uploaded = files.upload()

# Configure Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# ==========================================
# STEP 3: Download and Extract Dataset
# ==========================================
# Download dataset from Kaggle
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification

# Extract dataset
zip_file = '/content/fruits-fresh-and-rotten-for-classification.zip'
extract_path = '/content/fruit_dataset/'

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")
print("Directory structure:")
!ls -R /content/fruit_dataset/

# ==========================================
# STEP 4: Data Exploration and Analysis
# ==========================================

# Set dataset paths (adjust based on actual structure)
base_path = '/content/fruit_dataset/dataset/'
train_path = os.path.join(base_path, 'train')
test_path = os.path.join(base_path, 'test')

# Function to count images in each category
def analyze_dataset(path):
    """Analyze dataset structure and count images"""
    categories = {}
    if os.path.exists(path):
        for category in os.listdir(path):
            category_path = os.path.join(path, category)
            if os.path.isdir(category_path):
                num_images = len([f for f in os.listdir(category_path)
                                if f.endswith(('.jpg', '.jpeg', '.png'))])
                categories[category] = num_images
    return categories

# Analyze train and test sets
train_categories = analyze_dataset(train_path)
test_categories = analyze_dataset(test_path)

print("\n=== DATASET ANALYSIS ===")
print(f"\nTraining Set:")
for cat, count in train_categories.items():
    print(f"  {cat}: {count} images")

print(f"\nTest Set:")
for cat, count in test_categories.items():
    print(f"  {cat}: {count} images")

total_train = sum(train_categories.values())
total_test = sum(test_categories.values())
print(f"\nTotal Training Images: {total_train}")
print(f"Total Test Images: {total_test}")

# ==========================================
# STEP 5: Data Visualization
# ==========================================

# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Training set distribution
axes[0].bar(train_categories.keys(), train_categories.values(), color='skyblue')
axes[0].set_title('Training Set Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Categories')
axes[0].set_ylabel('Number of Images')
axes[0].tick_params(axis='x', rotation=45)

# Test set distribution
axes[1].bar(test_categories.keys(), test_categories.values(), color='lightcoral')
axes[1].set_title('Test Set Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Categories')
axes[1].set_ylabel('Number of Images')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/class_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# Display sample images from each category
def display_sample_images(path, categories, samples_per_category=3):
    """Display sample images from each category"""
    num_categories = len(categories)
    fig, axes = plt.subplots(num_categories, samples_per_category,
                            figsize=(15, 5*num_categories))

    for idx, category in enumerate(categories):
        category_path = os.path.join(path, category)
        images = [f for f in os.listdir(category_path)
                 if f.endswith(('.jpg', '.jpeg', '.png'))][:samples_per_category]

        for i, img_name in enumerate(images):
            img_path = os.path.join(category_path, img_name)
            img = Image.open(img_path)

            if num_categories == 1:
                axes[i].imshow(img)
                axes[i].set_title(f'{category}')
                axes[i].axis('off')
            else:
                axes[idx, i].imshow(img)
                axes[idx, i].set_title(f'{category}')
                axes[idx, i].axis('off')

    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/sample_images.png', dpi=300, bbox_inches='tight')
    plt.show()

print("\n=== SAMPLE IMAGES ===")
display_sample_images(train_path, list(train_categories.keys()))

# ==========================================
# STEP 6: Image Analysis (Size, Color Distribution)
# ==========================================

def analyze_image_properties(path, categories, num_samples=50):
    """Analyze image properties like dimensions and color distribution"""
    widths, heights, channels = [], [], []

    for category in categories:
        category_path = os.path.join(path, category)
        images = [f for f in os.listdir(category_path)
                 if f.endswith(('.jpg', '.jpeg', '.png'))][:num_samples]

        for img_name in images:
            img_path = os.path.join(category_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                h, w, c = img.shape
                heights.append(h)
                widths.append(w)
                channels.append(c)

    return widths, heights, channels

widths, heights, channels = analyze_image_properties(train_path, list(train_categories.keys()))

print("\n=== IMAGE PROPERTIES ANALYSIS ===")
print(f"Image Width - Mean: {np.mean(widths):.2f}, Std: {np.std(widths):.2f}")
print(f"Image Height - Mean: {np.mean(heights):.2f}, Std: {np.std(heights):.2f}")
print(f"Most common number of channels: {max(set(channels), key=channels.count)}")

# Visualize image dimensions
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].hist(widths, bins=30, color='blue', alpha=0.7)
axes[0].set_title('Image Width Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Width (pixels)')
axes[0].set_ylabel('Frequency')

axes[1].hist(heights, bins=30, color='green', alpha=0.7)
axes[1].set_title('Image Height Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Height (pixels)')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/image_dimensions.png', dpi=300, bbox_inches='tight')
plt.show()

# ==========================================
# STEP 7: Data Preprocessing and Augmentation
# ==========================================

# Image dimensions for ResNet50
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
NUM_CLASSES = len(train_categories)

# Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

# Only rescaling for validation and test
test_datagen = ImageDataGenerator(
    rescale=1./255
)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print("\n=== DATA GENERATORS ===")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Test samples: {test_generator.samples}")
print(f"Class indices: {train_generator.class_indices}")

# ==========================================
# STEP 8: Build ResNet50 Model
# ==========================================

def create_resnet50_model(num_classes, input_shape=(224, 224, 3)):
    """
    Create ResNet50 model with transfer learning
    """
    # Load pre-trained ResNet50 without top layers
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    # Freeze base model layers initially
    base_model.trainable = False

    # Create new model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model, base_model

# Create model
resnet_model, base_model = create_resnet50_model(NUM_CLASSES)

# Display model architecture
print("\n=== MODEL ARCHITECTURE ===")
resnet_model.summary()

# Save model architecture visualization
tf.keras.utils.plot_model(
    resnet_model,
    to_file='/content/drive/MyDrive/resnet50_architecture.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB',
    dpi=150
)

# ==========================================
# STEP 9: Compile Model
# ==========================================

# Compile model
initial_learning_rate = 0.001

resnet_model.compile(
    optimizer=Adam(learning_rate=initial_learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

print("\n=== MODEL COMPILED ===")
print(f"Optimizer: Adam")
print(f"Learning Rate: {initial_learning_rate}")
print(f"Loss Function: Categorical Crossentropy")

# ==========================================
# STEP 10: Setup Callbacks
# ==========================================

# Early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Reduce learning rate on plateau
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

# Model checkpoint
checkpoint_path = '/content/drive/MyDrive/resnet50_best_model.h5'
model_checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

callbacks = [early_stopping, reduce_lr, model_checkpoint]

# ==========================================
# STEP 11: Train Model (Phase 1 - Frozen Base)
# ==========================================

print("\n=== TRAINING PHASE 1: Frozen Base Model ===")
EPOCHS_PHASE1 = 20

history_phase1 = resnet_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS_PHASE1,
    callbacks=callbacks,
    verbose=1
)

# ==========================================
# STEP 12: Fine-tuning (Phase 2 - Unfrozen Base)
# ==========================================

print("\n=== TRAINING PHASE 2: Fine-tuning ===")

# Unfreeze the base model
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all layers before fine_tune_at
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile with lower learning rate
resnet_model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

EPOCHS_PHASE2 = 20

history_phase2 = resnet_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS_PHASE2,
    initial_epoch=history_phase1.epoch[-1],
    callbacks=callbacks,
    verbose=1
)

# ==========================================
# STEP 13: Training History Visualization
# ==========================================

def plot_training_history(history1, history2):
    """Plot training and validation metrics"""

    # Get precision/recall keys (handle both 'precision' and 'precision_1' naming)
    precision_key1 = 'precision' if 'precision' in history1.history else 'precision_1'
    recall_key1 = 'recall' if 'recall' in history1.history else 'recall_1'
    precision_key2 = 'precision' if 'precision' in history2.history else 'precision_1'
    recall_key2 = 'recall' if 'recall' in history2.history else 'recall_1'

    # Combine histories
    acc = history1.history['accuracy'] + history2.history['accuracy']
    val_acc = history1.history['val_accuracy'] + history2.history['val_accuracy']
    loss = history1.history['loss'] + history2.history['loss']
    val_loss = history1.history['val_loss'] + history2.history['val_loss']

    epochs_range = range(len(acc))

    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # Accuracy
    axes[0, 0].plot(epochs_range, acc, label='Training Accuracy', linewidth=2)
    axes[0, 0].plot(epochs_range, val_acc, label='Validation Accuracy', linewidth=2)
    axes[0, 0].axvline(x=len(history1.history['accuracy']), color='r',
                       linestyle='--', label='Fine-tuning starts')
    axes[0, 0].set_title('Training and Validation Accuracy', fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Loss
    axes[0, 1].plot(epochs_range, loss, label='Training Loss', linewidth=2)
    axes[0, 1].plot(epochs_range, val_loss, label='Validation Loss', linewidth=2)
    axes[0, 1].axvline(x=len(history1.history['loss']), color='r',
                       linestyle='--', label='Fine-tuning starts')
    axes[0, 1].set_title('Training and Validation Loss', fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Precision
    precision = history1.history[precision_key1] + history2.history[precision_key2]
    val_precision = history1.history['val_' + precision_key1] + history2.history['val_' + precision_key2]
    axes[1, 0].plot(epochs_range, precision, label='Training Precision', linewidth=2)
    axes[1, 0].plot(epochs_range, val_precision, label='Validation Precision', linewidth=2)
    axes[1, 0].axvline(x=len(history1.history['accuracy']), color='r',
                       linestyle='--', label='Fine-tuning starts')
    axes[1, 0].set_title('Training and Validation Precision', fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Recall
    recall = history1.history[recall_key1] + history2.history[recall_key2]
    val_recall = history1.history['val_' + recall_key1] + history2.history['val_' + recall_key2]
    axes[1, 1].plot(epochs_range, recall, label='Training Recall', linewidth=2)
    axes[1, 1].plot(epochs_range, val_recall, label='Validation Recall', linewidth=2)
    axes[1, 1].axvline(x=len(history1.history['accuracy']), color='r',
                       linestyle='--', label='Fine-tuning starts')
    axes[1, 1].set_title('Training and Validation Recall', fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_training_history(history_phase1, history_phase2)

# ==========================================
# STEP 14: Model Evaluation on Test Set
# ==========================================

print("\n=== MODEL EVALUATION ===")

# Load best model
resnet_model.load_weights(checkpoint_path)

# Evaluate on test set
test_loss, test_accuracy, test_precision, test_recall = resnet_model.evaluate(
    test_generator,
    verbose=1
)

print(f"\nTest Accuracy: {test_accuracy*100:.2f}%")
print(f"Test Precision: {test_precision*100:.2f}%")
print(f"Test Recall: {test_recall*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

# Calculate F1 Score
test_f1 = 2 * (test_precision * test_recall) / (test_precision + test_recall)
print(f"Test F1-Score: {test_f1*100:.2f}%")

# ==========================================
# STEP 15: Predictions and Confusion Matrix
# ==========================================

# Get predictions
test_generator.reset()
predictions = resnet_model.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix - ResNet50', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# ==========================================
# STEP 16: Classification Report
# ==========================================

# Generate classification report
report = classification_report(true_classes, predicted_classes,
                              target_names=class_labels, digits=4)
print("\n=== CLASSIFICATION REPORT ===")
print(report)

# Save report to file
with open('/content/drive/MyDrive/classification_report.txt', 'w') as f:
    f.write("=" * 50 + "\n")
    f.write("RESNET50 CLASSIFICATION REPORT\n")
    f.write("=" * 50 + "\n\n")
    f.write(report)
    f.write("\n\n" + "=" * 50 + "\n")
    f.write(f"Test Accuracy: {test_accuracy*100:.2f}%\n")
    f.write(f"Test Precision: {test_precision*100:.2f}%\n")
    f.write(f"Test Recall: {test_recall*100:.2f}%\n")
    f.write(f"Test F1-Score: {test_f1*100:.2f}%\n")
    f.write(f"Test Loss: {test_loss:.4f}\n")

# ==========================================
# STEP 17: Per-Class Performance Analysis
# ==========================================

# Calculate per-class metrics
from sklearn.metrics import precision_recall_fscore_support

precision_per_class, recall_per_class, f1_per_class, support = precision_recall_fscore_support(
    true_classes, predicted_classes, labels=range(NUM_CLASSES)
)

# Create DataFrame for better visualization
metrics_df = pd.DataFrame({
    'Class': class_labels,
    'Precision': precision_per_class,
    'Recall': recall_per_class,
    'F1-Score': f1_per_class,
    'Support': support
})

print("\n=== PER-CLASS PERFORMANCE ===")
print(metrics_df.to_string(index=False))

# Visualize per-class metrics
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(class_labels))
width = 0.25

bars1 = ax.bar(x - width, precision_per_class, width, label='Precision', alpha=0.8)
bars2 = ax.bar(x, recall_per_class, width, label='Recall', alpha=0.8)
bars3 = ax.bar(x + width, f1_per_class, width, label='F1-Score', alpha=0.8)

ax.set_xlabel('Classes', fontweight='bold')
ax.set_ylabel('Scores', fontweight='bold')
ax.set_title('Per-Class Performance Metrics - ResNet50', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(class_labels, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/per_class_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

# ==========================================
# STEP 18: Sample Predictions Visualization
# ==========================================

def display_predictions(generator, model, num_samples=12):
    """Display sample predictions with true and predicted labels"""
    generator.reset()

    # Get a batch of images
    images, labels = next(generator)
    predictions = model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(labels, axis=1)

    # Get class names
    class_names = list(generator.class_indices.keys())

    # Plot
    num_samples = min(num_samples, len(images))
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    axes = axes.ravel()

    for i in range(num_samples):
        axes[i].imshow(images[i])

        true_label = class_names[true_classes[i]]
        pred_label = class_names[predicted_classes[i]]
        confidence = predictions[i][predicted_classes[i]] * 100

        color = 'green' if true_classes[i] == predicted_classes[i] else 'red'

        axes[i].set_title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.1f}%',
                         color=color, fontweight='bold')
        axes[i].axis('off')

    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/sample_predictions.png', dpi=300, bbox_inches='tight')
    plt.show()

print("\n=== SAMPLE PREDICTIONS ===")
display_predictions(test_generator, resnet_model, num_samples=12)

# ==========================================
# STEP 19: Save Final Model
# ==========================================

# Save complete model
final_model_path = '/content/drive/MyDrive/resnet50_final_model.h5'
resnet_model.save(final_model_path)
print(f"\nFinal model saved to: {final_model_path}")

# Save model in SavedModel format (for deployment)
saved_model_path = '/content/drive/MyDrive/resnet50_savedmodel'
resnet_model.save(saved_model_path)
print(f"SavedModel format saved to: {saved_model_path}")

# ==========================================
# STEP 20: Model Summary Statistics
# ==========================================

print("\n" + "="*60)
print("RESNET50 MODEL - FINAL SUMMARY")
print("="*60)
print(f"\nDataset Statistics:")
print(f"  - Training Samples: {train_generator.samples}")
print(f"  - Validation Samples: {validation_generator.samples}")
print(f"  - Test Samples: {test_generator.samples}")
print(f"  - Number of Classes: {NUM_CLASSES}")
print(f"  - Classes: {', '.join(class_labels)}")

print(f"\nModel Architecture:")
print(f"  - Base Model: ResNet50 (ImageNet pre-trained)")
print(f"  - Input Shape: {IMG_HEIGHT}x{IMG_WIDTH}x3")
print(f"  - Total Parameters: {resnet_model.count_params():,}")
print(f"  - Trainable Parameters: {sum([tf.size(w).numpy() for w in resnet_model.trainable_weights]):,}")

print(f"\nTraining Configuration:")
print(f"  - Phase 1 Epochs: {EPOCHS_PHASE1}")
print(f"  - Phase 2 Epochs: {EPOCHS_PHASE2}")
print(f"  - Batch Size: {BATCH_SIZE}")
print(f"  - Initial Learning Rate: {initial_learning_rate}")
print(f"  - Fine-tuning Learning Rate: 1e-5")

print(f"\nFinal Test Performance:")
print(f"  - Accuracy: {test_accuracy*100:.2f}%")
print(f"  - Precision: {test_precision*100:.2f}%")
print(f"  - Recall: {test_recall*100:.2f}%")
print(f"  - F1-Score: {test_f1*100:.2f}%")
print(f"  - Loss: {test_loss:.4f}")

print("\n" + "="*60)
print("All results saved to Google Drive!")
print("="*60)

# Save summary to file
with open('/content/drive/MyDrive/model_summary.txt', 'w') as f:
    f.write("="*60 + "\n")
    f.write("RESNET50 MODEL - FINAL SUMMARY\n")
    f.write("="*60 + "\n\n")

    f.write("Dataset Statistics:\n")
    f.write(f"  - Training Samples: {train_generator.samples}\n")
    f.write(f"  - Validation Samples: {validation_generator.samples}\n")
    f.write(f"  - Test Samples: {test_generator.samples}\n")
    f.write(f"  - Number of Classes: {NUM_CLASSES}\n")
    f.write(f"  - Classes: {', '.join(class_labels)}\n\n")

    f.write("Model Architecture:\n")
    f.write(f"  - Base Model: ResNet50 (ImageNet pre-trained)\n")
    f.write(f"  - Input Shape: {IMG_HEIGHT}x{IMG_WIDTH}x3\n")
    f.write(f"  - Total Parameters: {resnet_model.count_params():,}\n\n")

    f.write("Training Configuration:\n")
    f.write(f"  - Phase 1 Epochs: {EPOCHS_PHASE1}\n")
    f.write(f"  - Phase 2 Epochs: {EPOCHS_PHASE2}\n")
    f.write(f"  - Batch Size: {BATCH_SIZE}\n")
    f.write(f"  - Initial Learning Rate: {initial_learning_rate}\n")
    f.write(f"  - Fine-tuning Learning Rate: 1e-5\n\n")

    f.write("Final Test Performance:\n")
    f.write(f"  - Accuracy: {test_accuracy*100:.2f}%\n")
    f.write(f"  - Precision: {test_precision*100:.2f}%\n")
    f.write(f"  - Recall: {test_recall*100:.2f}%\n")
    f.write(f"  - F1-Score: {test_f1*100:.2f}%\n")
    f.write(f"  - Loss: {test_loss:.4f}\n")

print("\n✅ ResNet50 implementation completed successfully!")
print("📁 Check your Google Drive for all saved files and visualizations.")

Output hidden; open in https://colab.research.google.com to view.